# Speciation Microservice example

This notebook performs a speciation calculation using a Docker container running the speciate function from [AqEquil](https://github.com/worm-portal/AqEquil).

## Step 0: Deploy the microservice

For cloud deployment, follow the instructions in [deployment-notes.md](https://github.com/jedick/speciation-microservice/blob/main/deployment-notes.md).

For local testing, follow the instructions in [README.md](https://github.com/jedick/speciation-microservice/blob/main/README.md).

## Step 1: Load the data

This downloads the CSV file for an aqueous speciation example from [WORM-Library](https://github.com/worm-portal/WORM-Library/tree/master/3-Aqueous-Speciation/1-Introduction-to-Aq-Speciation).
The data in this file are from [Leong et al., 2021](10.1029/2020JB020756).

If you would rather process a locally downloaded file, uncomment the bottom lines.

In [1]:
import json

# Read the CSV file (remote file)
import requests
# URL of the remote file
url = "https://raw.githubusercontent.com/worm-portal/WORM-Library/refs/heads/master/3-Aqueous-Speciation/1-Introduction-to-Aq-Speciation/leong2021.csv"
# Fetch the file content
response = requests.get(url)
# Put the file content into a JSON object
json_data = json.dumps({"input": response.text})

# # Read the CSV file (local file)
# with open("leong2021.csv", "r") as file:
#     lines = file.readlines()
# # Convert the lines from the CSV file to JSON as a single long string in a key named 'input'
# json_data = json.dumps({"input": "".join(lines)})

## Step 2: Run speciation calculation

This sends the data to the API endpoint and stores the response.
The speciation calculation takes some time depending on the size and complexity of the input file.

NOTE: If the calculation exceeds the timeout value set in AWS Lambda, the result will be 502 (Internal Server Error).

In [2]:
import requests

# This is the URL endpoint for the local version
url = "http://localhost:9000/2015-03-31/functions/function/invocations"

# If you have an AWS Lambda Function URL, put it here
#url = "https://<url-id>.lambda-url.<region>.on.aws/"

response = requests.post(url, json=json_data)

# Print the status code
print(f"Response status code: {response.status_code}")
# Print the error message for an unsuccessful request
if not response.status_code == 200:
    print(response.text)

Response status code: 200


## Step 3: Process the result

This converts the result to a Pandas DataFrame, using StringIO to simulate a file-like object.

In [3]:
import pandas as pd
from io import StringIO

try:
    output = response.json() # Local
except:
    output = response.text   # AWS Lambda Function URL

df = pd.read_csv(StringIO(output))
df

,Year,Area,pH_(input),Temperature,logfO2,HCO3-_(input),SiO2_(input),F-_(input),Cl-_(input),Br-_(input),...,K+_total,Li+_total,Mg+2_total,NH4+_total,NO2-_total,NO3-_total,Na+_total,OH-_total,SO4-2_total,SiO2_total
0,NaN,NaN,pH,degC,logfO2,Molality,Molality,Molality,Molality,Molality,...,molality,molality,molality,molality,molality,molality,molality,molality,molality,molality
1,NaN,NaN,7.2,33.8,-6.0,NaN,0.000374571,8.22e-06,0.001152699,NaN,...,3.72e-05,4.18e-06,0.00127299,NaN,NaN,NaN,0.001261,2.61703e-06,0.000371236,0.000374571
2,NaN,NaN,7.6,34.0,-6.0,NaN,0.000366025,4.49e-06,0.000758036,NaN,...,3.86e-05,4.32e-06,0.00112281,NaN,3.89e-06,0.0,0.000776082,6.40042e-06,0.000254701,0.000366025
3,NaN,NaN,7.7,21.0,-6.0,0.004391597,0.0002,NaN,0.00319,1.8e-06,...,0.000114,5.62e-07,0.00201,3.05e-07,2.2e-07,0.000116,0.0032,NaN,0.000443,0.0002
4,NaN,NaN,7.7,25.0,-6.0,0.00521696,0.000309412,4.34e-07,0.001280964,2.44e-07,...,8.07e-05,8.64e-07,0.00290804,2.35e-06,1.41e-07,0.00013386,0.001001,NaN,0.000645792,0.000309412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,NaN,11.8,18.0,-6.0,0.000124981,3.49e-06,6.47e-07,0.008534362,3.83e-06,...,0.000269812,2.12e-06,1.86e-06,NaN,NaN,NaN,0.0100229,0.00420885,1.91e-05,3.49e-06
135,NaN,NaN,11.8,22.4,-6.0,0.000102602,3.77e-06,1.26e-06,0.005322442,NaN,...,0.000125342,1.58e-05,5.35e-07,1.17e-05,NaN,4.68e-05,0.0070274,0.00586751,3.71e-06,3.77e-06
136,NaN,NaN,11.9,18.9,-6.0,6.89e-05,7.84e-06,NaN,0.013442481,5.49e-06,...,0.000294854,3.01e-05,1.46e-06,4.4e-05,NaN,2.58e-05,0.0182814,0.00572067,1.28e-05,7.84e-06
137,NaN,NaN,11.9,24.2,-6.0,9.55e-05,2.49e-06,NaN,0.011032503,NaN,...,0.000217793,2.19e-05,1.74e-06,NaN,NaN,NaN,0.0154081,0.00867614,5.72e-06,2.49e-06
